In [1]:
from pathlib import Path
import polars as pl
import ind_pl as ind
from pprint import pprint
from continuous import components
import seaborn as sns
from binance import Client
from mt.resources import keys
client = Client(keys.mPkey, keys.mSkey)

In [2]:
def choose_by_length(minimum: int|str=27500, maximum:int|str=420000):
    """checks the length of ohlc history of each trading pair, then makes a list of all pairs whos history length falls
    within the stated range"""

    lengths = {'1 month': 8750, '2 months': 17500, '3 months': 26250, '6 months': 52500,
               '1 year': 105000, '2 years': 210000, '3 years': 315000, '4 years': 420000}

    if isinstance(minimum, str):
        minimum = lengths[minimum]
    if isinstance(maximum, str):
        maximum = lengths[maximum]

    info = {}
    data_path = Path("/home/ross/coding/modular_trader/bin_ohlc_5m")
    for pair_path in list(data_path.glob('*')):
        df = pl.read_parquet(pair_path)
        info[pair_path.stem] = len(df)

    return [p for p, v in info.items() if minimum < v <= maximum]

def resample(df, timeframe):
    df = df.sort('timestamp').set_sorted('timestamp')
    
    df = (df.group_by_dynamic(pl.col('timestamp'), every=timeframe).agg(
        pl.first('open'),
        pl.max('high'),
        pl.min('low'),
        pl.last('close'),
        pl.sum('base_vol'),
        pl.sum('quote_vol'),
        pl.sum('num_trades'),
        pl.sum('taker_buy_base_vol'),
        pl.sum('taker_buy_quote_vol'),
        ))

    df = df.sort('timestamp')
    
    return df

def top_heavy(df: pl.DataFrame) -> pl.DataFrame:
    """calculates my 'top heavy' metric, which is a ratio representing the balance of total historic
    volume below the current price relative to total historic volume above the current price"""
    
    current_price = df.item(-1, 'close')
    
    above = df.filter(pl.col('close') > current_price)['base_vol'].sum()
    below = df.filter(pl.col('close') < current_price)['base_vol'].sum()
    
    return above, below, above / below

    

In [3]:
# top_heavy refers to an analysis i just thought up: is there more historic volume above or below current price? 
# this should be calculated as total volume above current price / total volume below

# to analyse the effect of daily/weekly rsi on a pair's suitablility for selection, i could go through ohlc history calculating rolling sharpe alongside rsi, then calculate the forward-looking difference between rsi values and rsi values 1 week/month ahead (so the diff in each period is a projection of how the rsi will change over the coming week/month), then compare each diff to it's corresponding sharpe value to see if there is a 'good' range of rsi values to select for (is it better to select overbought pairs, oversold pairs, or something else). it might even turn out to be a combination of daily and weekly that is most useful, like if one is high and the other is low then good, but if both high or both low then bad

# it seems that the shortest history that works with the ichi trend strategy is 27500 5min periods

idea: an allocator function that chooses what proportion of capital to allocate to trend-following vs mean-reversion. calculate a price channel based on a rolling window, then analyse where in the channel price spends most of the time. my guess is during trending regimes price will mostly be at the edges of the channel, and during ranging conditions price will spend most of the time in the middle of the channel. 
so an indicator could be created by calculating the distance from the middle of the channel to the (smoothed) current price, so if price is close to the middle, more weight is given to the mean-reverting forecasts, and if price is out at the extremes, more weight is given to the trend-following forecasts. 

i could also investigate whether vwma can tell me anything useful here, like if price is at the edge of the channel and 25h vwma is further out than 25h ema then continuation is more likely, but if ema is further out than vwma then mean-reversion is more likely etc. i could investigate this same idea with rsi too, ie when rsi is t the extremes, can the relationship between vwma and a non-volume ma have any predictive power about continuation or reversion? 

this will probably hurt overall profitability but might improve sharpe ratio.

i could even use machine learning here maybe, by looking at the channel position at time t, t-1, t-2, t-3 etc, can i get a solid probability on t+1's price action being either continuation or reversal? could vwma/ema ratio be a useful feature here? or volume delta? or volume z-score? or num_trades z-score?

In [6]:
x_info = client.get_exchange_info()
tick_sizes = {x['symbol']: float(x['filters'][0]['tickSize']) for x in x_info['symbols']}

info = {
    'pair': [],
    'length': [],
    'daily_volume': [],
    'weekly_volume': [],
    'daily_volume_change': [],
    'daily_atr': [],
    'tick_size': [],
    # 'weekly_rsi': [],
    # 'monthly_rsi': [],
    'volume_above_pw': [],  # historic volume above the current price, in multiples of current weekly volume
    'volume_below_pw': [],  # historic volume below the current price, in multiples of current weekly volume
    'top_heavy': [],
    # 'sharpe': [],  # this will have to be backtested
    # 'mcap': [],  # this will have to come from coingecko
    'divisibility': []
}

pairs = choose_by_length()

for pair in pairs:
    pair_path = Path(f"/home/ross/coding/pi_3/modular_trader/bin_ohlc_5m/{pair}.parquet")
    try:
        df = pl.read_parquet(pair_path)
    except FileNotFoundError:
        continue
    weekly_df = resample(df, '1w')
    monthly_df = resample(df, '1m')
    one_day_volume = df['quote_vol'].tail(288).sum()
    seven_day_volume = df['quote_vol'].tail(2016).sum()
    daily_volume_change = df['quote_vol'].ewm_mean(span=288).pct_change(288)[-1]
    daily_atr_pct = ind.atr(df, 48, 288, 'pct').item(-1, 'atr_48_288_pct')
    daily_atr_abs = ind.atr(df, 48, 288, 'abs').item(-1, 'atr_48_288_abs')
    info['pair'].append(pair)
    info['length'].append(len(df))
    info['daily_volume'].append(one_day_volume)
    info['weekly_volume'].append(seven_day_volume)
    info['daily_volume_change'].append(daily_volume_change)
    info['daily_atr'].append(daily_atr_pct)
    info['tick_size'].append(tick_sizes[pair])
    # info['weekly_rsi'].append(weekly_df['close'])
    # info['monthly_rsi'].append(monthly_df['close'])
    info['volume_above_pw'].append(top_heavy(df)[0] / seven_day_volume)
    info['volume_below_pw'].append(top_heavy(df)[1] / seven_day_volume)
    info['top_heavy'].append(top_heavy(df)[2])
    info['divisibility'].append(daily_atr_abs / tick_sizes[pair])

info_df = pl.from_dict(info)

MBLUSDT finished
KDAUSDT finished
FUNUSDT finished
DOGEUSDT finished
MATICUSDT finished
COMBOUSDT finished
OSMOUSDT finished
ARKUSDT finished
KMDUSDT finished
LEVERUSDT finished
PEPEUSDT finished
BNXUSDT finished
C98USDT finished
TFUELUSDT finished
BICOUSDT finished
PHAUSDT finished
LITUSDT finished
SHIBUSDT finished
ONEUSDT finished
CRVUSDT finished
TRUUSDT finished
ATOMUSDT finished
NTRNUSDT finished
VOXELUSDT finished
QUICKUSDT finished
FILUSDT finished
DYMUSDT finished
ALTUSDT finished
MEMEUSDT finished
IDUSDT finished
HIGHUSDT finished
PUNDIXUSDT finished
ILVUSDT finished
ASTRUSDT finished
ICPUSDT finished
XAIUSDT finished
FORUSDT finished
TIAUSDT finished
LQTYUSDT finished
KLAYUSDT finished
DUSKUSDT finished
BATUSDT finished
IDEXUSDT finished
STRAXUSDT finished
AUCTIONUSDT finished
XVSUSDT finished
NMRUSDT finished
SNTUSDT finished
DARUSDT finished
JTOUSDT finished
PAXGUSDT finished
JASMYUSDT finished
MDXUSDT finished
COSUSDT finished
CHESSUSDT finished
ASRUSDT finished
CAKEUSDT 

In [30]:
lively_pairs = info_df.filter(
    pl.col('divisibility').gt(80.0),
    pl.col('daily_volume').gt(2_500_000),
    pl.col('length').gt(27_500),
    pl.col('daily_atr').rank(descending=True).lt(50),
)

# load ohlc, resample to 1h, then calculate correlation matrix for all pairs in lively pairs, and record avg correlation as a new stat for each pair
all_closes = {}
for pair in lively_pairs['pair']:
    pair_path = Path(f"/home/ross/coding/pi_3/modular_trader/bin_ohlc_5m/{pair}.parquet")
    try:
        df = pl.read_parquet(pair_path)
    except FileNotFoundError:
        continue
    df = resample(df, '1h')
    all_closes[pair] = df['close']

min_length = min([len(x) for x in all_closes.values()])
all_closes = {k: v.tail(min_length) for k, v in all_closes.items()}
closes_df = pl.DataFrame(all_closes)

In [32]:
(
    lively_pairs
    .with_columns(
        pl.Series(closes_df.corr()
                  .mean()
                  .to_dicts()[0]
                  .values()
                  )
        .alias('avg_correlation')
    )
    .filter(
        pl.col('avg_correlation')
        .rank()
        .lt(16)
    )
    ['pair']
    .to_list()
)

['PEPEUSDT',
 'OPUSDT',
 'RAYUSDT',
 'JUPUSDT',
 'CREAMUSDT',
 'ARUSDT',
 'UNFIUSDT',
 'FRONTUSDT',
 'PENDLEUSDT',
 'ENSUSDT',
 'ARBUSDT',
 'LDOUSDT',
 'LTOUSDT',
 'ACEUSDT',
 'BLURUSDT']

In [24]:
dead_pairs = info_df.filter(
    pl.col('volume_above_pw').rank(descending=True) < 20,  # select the 10 pairs with the highest volume_above
    pl.col('top_heavy') > 1
)

likely_shorts = info_df.filter(
    pl.col('volume_above_pw').rank(descending=True) < 20,  # select the 10 pairs with the highest volume_above
    pl.col('top_heavy') < 1
)

In [31]:
dead_pairs

pair,length,daily_volume,weekly_volume,volume_above_pw,volume_below_pw,top_heavy
str,i64,f64,f64,f64,f64,f64
"""MBLUSDT""",420000,1.9593e6,2.3960e7,20245.136424,17680.237775,1.145072
"""FUNUSDT""",420000,1.7790e6,1.6734e7,15742.963768,3736.336984,4.213475
"""TROYUSDT""",420000,2.7175e6,2.7461e7,24187.215055,4544.342496,5.322489
"""1000SATSUSDT""",28514,5.3604e7,5.1728e8,28727.449875,3051.346916,9.414678
"""WINUSDT""",420000,3.2572e6,3.0805e7,1.6234e6,571936.668037,2.838421
"""HOTUSDT""",420000,8.6922e6,1.1362e8,27177.378656,20668.197702,1.314937
"""REEFUSDT""",338837,6.1148e6,6.2939e7,19231.562513,8750.034054,2.197884
"""DENTUSDT""",420000,4.8617e6,4.5726e7,89219.293618,49981.063421,1.785062
"""LUNCUSDT""",160738,2.7282e7,2.8572e8,193601.602484,163678.30882,1.182818


In [32]:
likely_shorts

pair,length,daily_volume,weekly_volume,volume_above_pw,volume_below_pw,top_heavy
str,i64,f64,f64,f64,f64,f64
"""PEPEUSDT""",92090,6.2235e8,5.1735e9,244054.769302,1.9212e6,0.12703
"""SHIBUSDT""",300880,4.0374e8,3.3201e9,1.0937e6,2.3391e6,0.467576
"""VTHOUSDT""",386247,2.8351e6,2.1915e7,27299.943938,32200.924105,0.8478
"""SPELLUSDT""",235330,4.4174e6,5.2628e7,21975.821573,52943.465335,0.415081
"""XECUSDT""",267538,1.0516e7,8.1596e7,604733.618449,727384.301537,0.831381
"""BONKUSDT""",27698,8.3368e7,1.1629e9,152789.566044,266073.716147,0.574238
"""SUNUSDT""",366027,3.0587e6,1.7495e7,13501.500557,14016.954354,0.963226
"""BTTCUSDT""",226162,9.2598e6,7.0179e7,1.6009e7,2.9953e7,0.534464


In [48]:
from binance import Client
from mt.resources import keys
client = Client(keys.mPkey, keys.mSkey)

info = client.get_exchange_info()
tick_sizes = {x['symbol']: float(x['filters'][0]['tickSize']) for x in info['symbols']}

In [55]:
tickers = client.get_ticker()
prices = {x['symbol']: float(x['weightedAvgPrice']) for x in tickers}

In [57]:
divisibility = {}
for p in lively_pairs['pair']:
    divisibility.update({p: (prices[p] / tick_sizes[p])})

divisibility

{'PEPEUSDT': 871.0,
 'MBOXUSDT': 3425.309,
 'HIGHUSDT': 4772.8081600000005,
 'NMRUSDT': 2738.5540469999996,
 'JTOUSDT': 4051.70762,
 'ASRUSDT': 3870.88796,
 'FTTUSDT': 18189.9539,
 'PEOPLEUSDT': 2946.066,
 'USTCUSDT': 2309627.0,
 'POWRUSDT': 3271.1607999999997,
 'POLYXUSDT': 3859.1838,
 'ARUSDT': 39922.71604,
 'LPTUSDT': 17516.86512,
 'WLDUSDT': 5800.926579999999,
 'XVGUSDT': 5927.18,
 'ARKMUSDT': 24795.867299999998,
 'KP3RUSDT': 7304.976023,
 'NFPUSDT': 4675.1404,
 'TRBUSDT': 13610.829673,
 'RAREUSDT': 1425.0394999999999,
 'UMAUSDT': 4363.30826,
 'FRONTUSDT': 14059.2658,
 'BTTCUSDT': 117.0,
 'PHBUSDT': 23645.2212,
 'QIUSDT': 2012.4929999999997,
 'MOVRUSDT': 15910.26581,
 'ARPAUSDT': 7226.348999999999,
 'AERGOUSDT': 1357.8269}

In [58]:
tickers

[{'symbol': 'ETHBTC',
  'priceChange': '-0.00065000',
  'priceChangePercent': '-1.357',
  'weightedAvgPrice': '0.04748249',
  'prevClosePrice': '0.04791000',
  'lastPrice': '0.04726000',
  'lastQty': '5.78320000',
  'bidPrice': '0.04726000',
  'bidQty': '36.13120000',
  'askPrice': '0.04727000',
  'askQty': '17.12500000',
  'openPrice': '0.04791000',
  'highPrice': '0.04802000',
  'lowPrice': '0.04709000',
  'volume': '17917.02160000',
  'quoteVolume': '850.74487710',
  'openTime': 1715511546194,
  'closeTime': 1715597946194,
  'firstId': 443732191,
  'lastId': 443807325,
  'count': 75135},
 {'symbol': 'LTCBTC',
  'priceChange': '-0.00002700',
  'priceChangePercent': '-2.026',
  'weightedAvgPrice': '0.00131630',
  'prevClosePrice': '0.00133400',
  'lastPrice': '0.00130600',
  'lastQty': '97.84900000',
  'bidPrice': '0.00130500',
  'bidQty': '183.11600000',
  'askPrice': '0.00130600',
  'askQty': '138.98000000',
  'openPrice': '0.00133300',
  'highPrice': '0.00134200',
  'lowPrice': '0.

In [13]:
client.get_symbol_info('1000SATSUSDT')

{'symbol': '1000SATSUSDT',
 'status': 'TRADING',
 'baseAsset': '1000SATS',
 'baseAssetPrecision': 2,
 'quoteAsset': 'USDT',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 2,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'otoAllowed': False,
 'quoteOrderQtyMarketAllowed': True,
 'allowTrailingStop': True,
 'cancelReplaceAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': True,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.00000010',
   'maxPrice': '1.00000000',
   'tickSize': '0.00000010'},
  {'filterType': 'LOT_SIZE',
   'minQty': '1.00',
   'maxQty': '36348600000.00',
   'stepSize': '1.00'},
  {'filterType': 'ICEBERG_PARTS', 'limit': 10},
  {'filterType': 'MARKET_LOT_SIZE',
   'minQty': '0.00',
   'maxQty': '1089337576.30',
   'stepSize': '0.00'},
  {'filterType': 'TRAILING_DELTA',
   'mi